In [5]:
import torch
import monai

In [6]:
from torch.utils.data import Dataset


class PairRegistrationDataset(Dataset):
    def __init__(self, X):
        self.X = X
        self.len = len(self.X)

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        index_moving = torch.randint(self.len, (1,))
        index_fixed = torch.randint(self.len, (1,))

        _moving = self.X[index_moving][0]
        _fixed = self.X[index_fixed][0]

        return _moving, _fixed


In [7]:
import pytorch_lightning as pl
from monai.networks.blocks import Warp
from monai.losses import LocalNormalizedCrossCorrelationLoss, MultiScaleLoss, DiceLoss, BendingEnergyLoss
import torch.nn.functional as F


class RegistrationUnetModel(pl.LightningModule):
    def __init__(self, weights):
        super().__init__()
        self.weights = weights
        self.model = monai.networks.nets.UNet(
            dimensions=3,
            in_channels=1,
            out_channels=1,
            channels=(16, 32, 64, 128, 256),
            strides=(2, 2, 2, 2),
            num_res_units=2,
        )
        self.warp = Warp("bilinear", "border")

    def forward(self, moving, fixed):
        x = torch.cat([moving, fixed], dim=1)
        ddf = self.model(x)
        pred_image = self.warp(moving, ddf)

        return pred_image, ddf

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.0001)
        #scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.9, patience=10, verbose=True)
        #optimizersdict = {"optimizer": optimizer, "lr_scheduler": {"scheduler": scheduler, "monitor": "loss""}}
        return optimizer

    def training_step(self, batch, batch_idx):
        source, tmp = batch
        y_source, y_target, forward_flow = self(source)
        loss = F.mse_loss(self.atlas, y_source) + F.mse_loss(y_target, source) +  0.01 * torch.norm(torch.mean(forward_flow, 0), p=2)
        return loss


In [ ]:
from monai.data import DataLoader, CacheDataset


In [8]:
transform = transforms.ToTensor()

train_data = datasets.MNIST(root='data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False, download=True, transform=transform)

In [1]:
print(train_data.data.shape)

NameError: name 'train_data' is not defined